# Welcome! #
## Enzyme and Zygote Performance Shootout ##

### Project Information ###

If you're interested in seeing a project synopsis and the benchmarks for performance we'll measuring, please take a look at our project documentation. 

### What Now? ###

Let's start by setting up Enzyme and Zygote in our environment to start testing some differentiation. Our primary benchmark is the physical timing of a differentiation operation. In order to measure this we will simply be using the `@time` Julia method. These times will then be compared to measure how fast differentiation is actually done, along with comparing other important benchmarks.

#### Import the Packages ####

In [1]:
using Pkg

Pkg.add("Enzyme")
Pkg.add("Zygote")
Pkg.add("Plots")

In [2]:
using Zygote
using Enzyme
using Plots
using Statistics

#### What Next? ####

Now that the packages have been added to the environment, we can start testing them out. First, a quick demonstration of the timing function we will be using. 

In [3]:
function fib(n)
    if n <= 1
        return 1
    else
        return fib(n - 1) + fib(n - 2)
    end
end
    
@time fib(20)

  0.000040 seconds


10946

#### Functions ####

For this shootout we will be handling differentiation for rootfinding problems. Specifically, we will be testing differentiation efficiency for aiding five different rootfinding algorithms, being Halley's, Golbabai-Javidi, Newton's, Noor's, and Zhanlav's Method.

Those methods look like such:

**Halley's Method**

$x_{n + 1} = x_{n} - \frac{2f(x_n)f^{\prime}(x_n)}{2f^{\prime}(x_n)^2 - f(x_n)f^{\prime \prime}(x_n)}$

**Golbabai-Javidi Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} - \frac{f(x_n)f^{\prime \prime}(x_n)}{2(f^{\prime \prime \prime}(x_n) - f(x_n)f^{\prime}(x_n)f^{\prime \prime}(x_n))}$

**Newton's Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)}$

**Noor's Method**

$y_n = x_n - \frac{f(x_n)}{f^{\prime}(x_n)}$

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} + (\frac{f(x_n)}{f^{\prime}(x_n)})\frac{f^{\prime}(y_n)}{f^{\prime}(x_n)}$

**Zhanlav's Method**

$z_n = y_n - \frac{f(y_n)}{f^{\prime}(y_n)}$

$q_n = z_n - \frac{f(z_n)}{f^{\prime}(y_n)}$

$y_{n + 1} = z_n - \frac{f(z_n) + f(q_n)}{f^{\prime}(y_n)}$

Before implementation of these various rootfinding algorithms, however, we can demonstrate some fairly basic differentiation using some basic functions below. This, similar to the time test, will present the methods we will be using and how they work.

We will be performing both of these tests using Enzyme and Zygote on three different functions, which will be defined below.

#### Test Functions ####

$f(x) = 5x^{10}$

$g(x) = 3x^3(cos(x) - 10x)$

$h(x) = e^{\frac{5x}{2}(sin(x)^{e^x})}$

#### Zygote Method ####

Zygote is the auto differentiation tool specifically made for Julia and uses the `gradient` method for computing derivatives. We hypothesize that this tool will likely be better optimized for Julia, but we will test this below by timing some basic differentiation!

#### Enzyme Method ####

Enzyme is another auto differentiation tool that is not specifically designed for Julia and is more generally designed for many different languages. This package uses the `autodiff` method for computing derivatives. Given that this tool is not specifically built for Julia, we hypothesize that these method calls will be significantly less optimized.

#### What are we testing? ####

For all three functions we will be timing the differentiation speed of both packages at an x-value of $2$.

#### Function One ####

In [4]:
f(x) = 5x^10
x = 2
@time zyg = gradient(x -> f(x), x)
@time enz = autodiff(f, Active(x))
@show zyg
@show enz

 25.728783 seconds (28.55 M allocations: 1.645 GiB, 6.51% gc time, 100.01% compilation time)
 18.823700 seconds (19.28 M allocations: 1.081 GiB, 4.28% gc time, 0.03% compilation time)
zyg = (25600.0,)
enz = (25600.0,)


(25600.0,)

#### Function Two ####

In [5]:
gf(x) = 3x^3 * (cos(x) - 10x)
x = 2
@time zyg = gradient(x -> gf(x), x)
@time enz = autodiff(gf, Active(x))
@show zyg
@show enz

  0.345368 seconds (493.11 k allocations: 28.390 MiB, 18.03% gc time, 99.93% compilation time)
  4.621116 seconds (6.26 M allocations: 362.781 MiB, 6.03% gc time, 39.40% compilation time)
zyg = (-996.8044243595134,)
enz = (-996.8044243595134,)


(-996.8044243595134,)

#### Function Three ####

In [6]:
hf(x) = exp((5x / 2) * (sin(x)^(exp(x))))
x = 2
@time zyg = gradient(x -> hf(x), x)
@time enz = autodiff(hf, Active(x))
@show zyg
@show enz

  0.390778 seconds (649.28 k allocations: 40.225 MiB, 20.22% gc time, 99.93% compilation time)
  2.651444 seconds (3.32 M allocations: 188.146 MiB, 6.47% gc time, 91.01% compilation time)
zyg = (-105.63101149036947,)
enz = (-105.63101149036945,)


(-105.63101149036945,)

#### What are the results? ####

From using the differentiation method from Zygote and Enzyme in the above functions, we can see that both methods produce essentially the same results, being identical in the first two cases and different at the $10^{-14}$ digit in the last results. Reasonably, we can conclude that both of these methods are possess similar accuarcy from this small test. We can also see that both methods have relatively identical speeds, with Enzyme being generally a bit slower and Zygote a bit faster. In terms of time, no conclusions can really be drawn, so it might be worth performing these computations a few times and pulling the respective means, variances, standard deviations, and medians. Finally, we can clearly see that Enzyme also generally requires more memory allocation than Zygote and takes a smaller percentage of compilation time. This similarly is a bit hard to gage however which is better, so a similar approach to the aforementioned would be useful.

#### Data Science ####

Let's try collecting a larger sample set of differentiation tests to better understand which method seems to be more optimal. Below we have defined a for loop that calls the differentiation methods $100$ times each, and then collects the results. These results will be passed to a data science function at the end that uses a variety of Julia Statistics methods to compute the corresponding data science values. For the tests, we will once again differentiate at $x = 2$.

In [7]:
function dataSci(timeEnz, sizeEnz, timeZyg, sizeZyg)
    timeEnzMean = mean(timeEnz)
    sizeEnzMean = mean(sizeEnz)
    timeZygMean = mean(timeZyg)
    sizeZygMean = mean(sizeZyg)
    
    timeEnzVar = var(timeEnz)
    sizeEnzVar = var(sizeEnz)
    timeZygVar = var(timeZyg)
    sizeZygVar = var(sizeZyg)
    
    timeEnzStd = timeEnzVar ^ (1 / 2)
    sizeEnzStd = sizeEnzVar ^ (1 / 2)
    timeZygStd = timeZygVar ^ (1 / 2)
    sizeZygStd = sizeZygVar ^ (1 / 2)
    
    timeEnzMed = median(timeEnz)
    sizeEnzMed = median(sizeEnz)
    timeZygMed = median(timeZyg)
    sizeZygMed = median(sizeZyg)
    
    @show timeEnzMean
    @show timeEnzVar
    @show timeEnzStd
    @show timeEnzMed
    
    @show sizeEnzMean
    @show sizeEnzVar
    @show sizeEnzStd
    @show sizeEnzMed
    
    @show timeZygMean
    @show timeZygVar
    @show timeZygStd
    @show timeZygMed
    
    @show sizeZygMean
    @show sizeZygVar
    @show sizeZygStd
    @show sizeZygMed
end

dataSci (generic function with 1 method)

#### Function One ####

In [13]:
timesEnz1 = []
timesZyg1 = []
sizesEnz1 = []
sizesZyg1 = []

for i in 1:100
    
    f(x) = 5x^10 # I don't understand a ton about Julia, but if I define this outside of the loop,
                 # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> f(x), x)

    push!(timesZyg1, tZyg)
    push!(sizesZyg1, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(f, Active(x))

    push!(timesEnz1, tEnz)
    push!(sizesEnz1, sizeEnz)

end

dataSci(timesEnz1, sizesEnz1, timesZyg1, sizesZyg1)

timeEnzMean = 0.09267851182999999
timeEnzVar = 0.002728936459141259
timeEnzStd = 0.05223922337804476
timeEnzMed = 0.083599264
sizeEnzMean = 5.5191422e6
sizeEnzVar = 4.909829171717173e7
sizeEnzStd = 7007.017319599811
sizeEnzMed = 5.518323e6
timeZygMean = 0.04129968049000002
timeZygVar = 0.00023591764811405283
timeZygStd = 0.01535961093628523
timeZygMed = 0.036597703999999995
sizeZygMean = 3.64246005e6
sizeZygVar = 1.7131734902499979e9
sizeZygStd = 41390.49999999997
sizeZygMed = 3.638321e6


3.638321e6

#### Results Function One ####

- **Zygote**

    - **Time**
    
        - Mean = 0.04123 seconds
        - Variance = 0.000236 seconds
        - Standard Deviation = 0.01535 seconds
        - Median = 0.0366 seconds
       
    - **Alloc. Size**
    
        - Mean = 3,642,460 bytes (3.64 MB)
        - Variance = 1,713,173,490 bytes (1.71 GB)
        - Standard Deviation = 41,390 bytes (41.39 KB)
        - Median = 3,638,321 bytes (3.64 MB)

- **Enzyme**

    - **Time**
    
        - Mean = 0.09267 seconds
        - Variance = 0.002728 seconds
        - Standard Deviation = 0.05224 seconds
        - Median = 0.0836 seconds
    
    - **Alloc. Size**
    
        - Mean = 5,519,142 bytes (5.52 MB)
        - Variance = 49,098,291 bytes (49.10 MB)
        - Standard Deviation = 7,007.02 bytes (7.01 KB)
        - Median = 5,518,323 bytes (5.52 MB)

#### Function Two ####

In [8]:
timesEnz2 = []
timesZyg2 = []
sizesEnz2 = []
sizesZyg2 = []

for i in 1:100
    
    gf(x) = 3x^3 * (cos(x) - 10x) # I don't understand a ton about Julia, but if I define this outside of the loop,
                                             # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> gf(x), x)

    push!(timesZyg2, tZyg)
    push!(sizesZyg2, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(gf, Active(x))

    push!(timesEnz2, tEnz)
    push!(sizesEnz2, sizeEnz)

end

dataSci(timesEnz2, sizesEnz2, timesZyg2, sizesZyg2)

timeEnzMean = 0.20542909254999991
timeEnzVar = 0.008447287654177004
timeEnzStd = 0.09190912715381973
timeEnzMed = 0.1756100965
sizeEnzMean = 6.91564444e6
sizeEnzVar = 540688.0064646468
sizeEnzStd = 735.3149029257103
sizeEnzMed = 6.915419e6
timeZygMean = 0.08563981077
timeZygVar = 0.00987911842820892
timeZygStd = 0.09939375447284865
timeZygMed = 0.0536120865
sizeZygMean = 6.02282301e6
sizeZygVar = 1.71847542667667e9
sizeZygStd = 41454.4982683022
sizeZygMed = 6.018635e6


6.018635e6

#### Results Function Two ####

- **Zygote**

    - **Time**
    
        - Mean = 0.08564 seconds
        - Variance = 0.00988 seconds
        - Standard Deviation = 0.09940 seconds
        - Median = 0.05361 seconds
       
    - **Alloc. Size**
    
        - Mean = 6,022,823 bytes (6.02 MB)
        - Variance = 1,718,475,426 bytes (1.72 GB)
        - Standard Deviation = 41,454 bytes (41.45 KB)
        - Median = 6,018,635 bytes (6.02 MB)

- **Enzyme**

    - **Time**
   
        - Mean = 0.20543 seconds
        - Variance = 0.00845 seconds
        - Standard Deviation = 0.09191 seconds
        - Median = 0.17561 seconds
    
    - **Alloc. Size**
    
        - Mean = 6,915,644 bytes (6.91 MB)
        - Variance = 540,688 bytes (540.69 KB)
        - Standard Deviation = 735.31 bytes (735.31 B)
        - Median = 6,915,419 bytes (6.91 MB)

#### Function Three ####

In [14]:
timesEnz3 = []
timesZyg3 = []
sizesEnz3 = []
sizesZyg3 = []

for i in 1:100
    
    hf(x) = exp((5x / 2) * (sin(x)^(exp(x)))) # I don't understand a ton about Julia, but if I define this outside of the loop,
                                              # the timed method doesn't work???
    
    valZyg, tZyg, sizeZyg = @timed gradient(x -> hf(x), x)

    push!(timesZyg3, tZyg)
    push!(sizesZyg3, sizeZyg)

    valEnz, tEnz, sizeEnz = @timed autodiff(hf, Active(x))

    push!(timesEnz3, tEnz)
    push!(sizesEnz3, sizeEnz)

end

dataSci(timesEnz3, sizesEnz3, timesZyg3, sizesZyg3)

timeEnzMean = 0.22160837142
timeEnzVar = 0.007148965185897377
timeEnzStd = 0.08455155342095955
timeEnzMed = 0.1992649405
sizeEnzMean = 8.28255332e6
sizeEnzVar = 4.9230193674343474e7
sizeEnzStd = 7016.423139630582
sizeEnzMed = 8.281633e6
timeZygMean = 0.09240678975000005
timeZygVar = 0.004842086234737579
timeZygStd = 0.06958510066628903
timeZygMed = 0.0739500035
sizeZygMean = 6.94505553e6
sizeZygVar = 1.7102608380899878e9
sizeZygStd = 41355.29999999985
sizeZygMed = 6.94092e6


6.94092e6

#### Results Function Three ####

- **Zygote**

    - **Time**
    
        - Mean = 0.09241 seconds
        - Variance = 0.00484 seconds
        - Standard Deviation = 0.06958 seconds
        - Median = 0.07395 seconds
       
    - **Alloc. Size**
    
        - Mean = 6,945,055 (6.94 MB)
        - Variance = 1,710,260,838 bytes (1.71 GB)
        - Standard Deviation = 41,355 bytes (41.35 KB)
        - Median = 6,940,920 bytes (6.94 MB)

- **Enzyme**

    - **Time**
    
        - Mean = 0.22160 seconds
        - Variance = 0.00715 seconds
        - Standard Deviation = 0.08455 seconds
        - Median = 0.19926 seconds
    
    - **Alloc. Size**
    
        - Mean = 8,282,553 bytes (8.28 MB)
        - Variance = 49,230,194 bytes (49.23 MB)
        - Standard Deviation = 7,016.42 bytes (7.02 KB)
        - Median = 8,281,633 bytes (8.28 MB)

#### Implications? ####



#### Practical Application ####

Theory is great and all, but what's a simple application for differentiation? There's a wide range of applications from gradient-descent to modeling certain processes, but for our project we decided some simple rootfinding algorithms would be useful. Thus, below are some rootfinding implementations of these algorithms using Enzyme and Zygote which produce a more noticeable time and memory difference between the two packages.

# Newton's with Enzyme #

In [18]:

    # testing Newton's with Enzyme

function newton(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newton(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7503638678402439  f(x)=-0.018923073822117442  f'(x)=-1.6819049529414878
[3] x=0.7391128909113617  f(x)=-4.6455898990771516e-5  f'(x)=-1.6736325442243012
[4] x=0.739085133385284  f(x)=-2.847205804457076e-10  f'(x)=-1.6736120293089505
[5] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 5)

# Newton's with Zygote #

In [8]:

    # testing Newton's with Zygote

function newton(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newton(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7503638678402439  f(x)=-0.018923073822117442  f'(x)=-1.6819049529414878
[3] x=0.7391128909113617  f(x)=-4.6455898990771516e-5  f'(x)=-1.6736325442243012
[4] x=0.739085133385284  f(x)=-2.847205804457076e-10  f'(x)=-1.6736120293089505
[5] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 5)

# Halley's with Enzyme #

In [9]:

    # halley's method with Enzyme

function halley(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halley(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7497462708604701  f(x)=-0.017884473924961064  f'(x)=-1.681453087296791
[3] x=0.7391097446115611  f(x)=-4.1190152908709976e-5  f'(x)=-1.6736302188963899
[4] x=0.7390851333479477  f(x)=-2.2223400897303236e-10  f'(x)=-1.6736120292813559
[5] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 5)

# Halley's with Zygote #

In [10]:

    # halley's method with Zygote

function halley(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halley(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7497462708604701  f(x)=-0.017884473924961064  f'(x)=-1.681453087296791
[3] x=0.7391097446115611  f(x)=-4.1190152908709976e-5  f'(x)=-1.6736302188963899
[4] x=0.7390851333479477  f(x)=-2.2223400897303236e-10  f'(x)=-1.6736120292813559
[5] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 5)

# Golbabai-Javidi's with Enzyme #

In [11]:

    # Golbabai-Javidi's method with Enzyme

function GJ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        fpppx = first(Enzyme.autodiff(f, Active(fppx)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7593355992363587  f(x)=-0.03404202857649352  f'(x)=-1.6884397114921996
[3] x=0.7392922883076712  f(x)=-0.0003467131120002964  f'(x)=-1.6737651199778294
[4] x=0.7390860657816581  f(x)=-1.5607548295992757e-6  f'(x)=-1.673612718428956
[5] x=0.7390851373583662  f(x)=-6.934118723656013e-9  f'(x)=-1.6736120322453965
[6] x=0.739085133233567  f(x)=-3.080502519736683e-11  f'(x)=-1.6736120291968186
[7] x=0.7390851332152425  f(x)=-1.3700152123874432e-13  f'(x)=-1.6736120291832752
[8] x=0.739085133215161  f(x)=-6.661338147750939e-16  f'(x)=-1.673612029183215


(0.739085133215161, -6.661338147750939e-16, 8)

# Golbabai-Javidi's with Zygote #

In [12]:

    # Golbabai-Javidi's method with Zygote

function GJ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        fpppx = first(Zygote.gradient(f, fppx))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.7593355992363587  f(x)=-0.03404202857649352  f'(x)=-1.6884397114921996
[3] x=0.7392922883076712  f(x)=-0.0003467131120002964  f'(x)=-1.6737651199778294
[4] x=0.7390860657816581  f(x)=-1.5607548295992757e-6  f'(x)=-1.673612718428956
[5] x=0.7390851373583662  f(x)=-6.934118723656013e-9  f'(x)=-1.6736120322453965
[6] x=0.739085133233567  f(x)=-3.080502519736683e-11  f'(x)=-1.6736120291968186
[7] x=0.7390851332152425  f(x)=-1.3700152123874432e-13  f'(x)=-1.6736120291832752
[8] x=0.739085133215161  f(x)=-6.661338147750939e-16  f'(x)=-1.673612029183215


(0.739085133215161, -6.661338147750939e-16, 8)

# Noor's with Enzyme #

In [13]:

    # Noor's method with Enzyme

function noor(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        y = x - fx/fpx
        fpy = first(Enzyme.autodiff(f, Active(y)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = cos(x) - x
noor(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.9783686806103187  f(x)=-0.41999206915629916  f'(x)=-1.8295875841531906
[3] x=0.9596967409291637  f(x)=-0.3859283539588626  f'(x)=-1.8190176054966698
[4] x=0.9434626910389852  f(x)=-0.3564745204608044  f'(x)=-1.8095955086345776
[5] x=0.9292518941995284  f(x)=-0.33081838273317876  f'(x)=-1.8011724735374115
[6] x=0.9167326606030312  f(x)=-0.30831624178299744  f'(x)=-1.7936179567715218
[7] x=0.9056378692749032  f(x)=-0.2884540514119285  f'(x)=-1.7868189975897142
[8] x=0.8957508924790036  f(x)=-0.270818105510059  f'(x)=-1.7806785585445701
[9] x=0.8868948123836067  f(x)=-0.2550728705729093  f'(x)=-1.7751135619033895
[10] x=0.8789241368419305  f(x)=-0.2409441519843235  f'(x)=-1.7700529455310767
[11] x=0.8717184072464472  f(x)=-0.22820623980557786  f'(x)=-1.7654358825894918
[12] x=0.8651772399280803  f(x)=-0.21667203089876896  f'(x)=-1.7612102166036983
[13] x=0.8592164568228965  f(x)=-0.20618538662881936  f'(x)=-1.7573311173

# Noor's with Zygote #

In [14]:

    # Noor's method with Zygote

function noor(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        y = x - fx/fpx
        fpy = first(Zygote.gradient(f, y))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = cos(x) - x
noor(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.9783686806103187  f(x)=-0.41999206915629916  f'(x)=-1.8295875841531906
[3] x=0.9596967409291637  f(x)=-0.3859283539588626  f'(x)=-1.8190176054966698
[4] x=0.9434626910389852  f(x)=-0.3564745204608044  f'(x)=-1.8095955086345776
[5] x=0.9292518941995284  f(x)=-0.33081838273317876  f'(x)=-1.8011724735374115
[6] x=0.9167326606030312  f(x)=-0.30831624178299744  f'(x)=-1.7936179567715218
[7] x=0.9056378692749032  f(x)=-0.2884540514119285  f'(x)=-1.7868189975897142
[8] x=0.8957508924790036  f(x)=-0.270818105510059  f'(x)=-1.7806785585445701
[9] x=0.8868948123836067  f(x)=-0.2550728705729093  f'(x)=-1.7751135619033895
[10] x=0.8789241368419305  f(x)=-0.2409441519843235  f'(x)=-1.7700529455310767
[11] x=0.8717184072464472  f(x)=-0.22820623980557786  f'(x)=-1.7654358825894918
[12] x=0.8651772399280803  f(x)=-0.21667203089876896  f'(x)=-1.7612102166036983
[13] x=0.8592164568228965  f(x)=-0.20618538662881936  f'(x)=-1.7573311173

# Zhanlav's with Enzyme #

In [15]:

    # Zhanlav's method with Enzyme

function zhanlav(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Enzyme.autodiff(f, Active(z)))

        q = z - fz/fpz
        fq = f(q)
        
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlav(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.740062576167659  f(x)=-0.0016362132372911287  f'(x)=-1.6743341208402764
[3] x=0.7390851333060271  f(x)=-1.5207513026638253e-10  f'(x)=-1.673612029250373
[4] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 4)

# Zhanlav's with Zygote #

In [16]:

    # Zhanlav's method with Zygote

function zhanlav(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Zygote.gradient(f, z))
        
        q = z - fz/fpz
        fq = f(q)
        
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlav(f, 1; tol=1e-15, verbose=true)

[1] x=1  f(x)=-0.45969769413186023  f'(x)=-1.8414709848078965
[2] x=0.740062576167659  f(x)=-0.0016362132372911287  f'(x)=-1.6743341208402764
[3] x=0.7390851333060271  f(x)=-1.5207513026638253e-10  f'(x)=-1.673612029250373
[4] x=0.7390851332151607  f(x)=0.0  f'(x)=-1.6736120291832148


(0.7390851332151607, 0.0, 4)

# Forward difference with Enzyme & Zygote #

In [17]:

    # testing
    # forward difference method with Enzyme and Zygote's functions.

f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
x = rand(5)
x2 = rand(5)

g = x -> Enzyme.fwddiff(f, x);
g2 = x2 -> Zygote.forwarddiff(f, x);

println(x)
println(x2)

[0.5048506721831889, 0.07769202306010259, 0.39605201208864704, 0.9446193536370395, 0.10167505278872202]
[0.9681102010341573, 0.02794158836224958, 0.7865703877490788, 0.5084142065330726, 0.08674916020906553]
